# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [5]:
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(  # 검증 세트 val_input, val_target 만들기
    train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))  # 과대적합 되었음
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [9]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)  # 객체로 모델과 훈련 세트 전체를 전달. 기본값 k==5
print(scores)

{'fit_time': array([0.00909376, 0.00648904, 0.00713611, 0.00542402, 0.0062511 ]), 'score_time': array([0.00107598, 0.00081205, 0.00061488, 0.00063109, 0.00053215]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
import numpy as np

print(np.mean(scores['test_score']))   # 교차 검증 최종점수 5개의 평균

0.855300214703487


- 회귀모델은 KFold 분할기
- 분류모델은 StratifiedKFold 사용

In [12]:
from sklearn.model_selection import StratifiedKFold  

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  # 훈련세트를 섞어서 10-폴드 교차 검증

scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
### GridSearchCV 클래스 : 하이퍼파라미터 검색과 교차 검증을 한번에 수행

In [14]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [15]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)  # n_jobs: 병렬수행할 CPU 코어 수

#### fit() 메서드를 호출하면 min_impurity_decrease 값마다 5(cv 매개변수 기본값)-폴드 교차 검증 수행
#### 5 x 5 = 25 개의 모델을 훈련

In [16]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

#### 25개 모델중 점수가 가장 높은 모델의 매개변수 조합으로 `전체 훈련 세트`로 다시 훈련 >> 이 모델은 best_estimator_ 속성에 저장됨


In [21]:
dt = gs.best_estimator_  # 최적의 매개변수를 갖는 모델
print(dt.score(train_input, train_target))

0.9615162593804117


#### 최적의 매개변수는 best_params_ 속성에 저장됨

In [22]:
print(gs.best_params_)  # 최적의 매개변수

{'min_impurity_decrease': 0.0001}


#### 각 매개변수에서의 교차 검증 평균점수는 cv_results_ 속성의 mean_test_score 키에 저장됨

In [24]:
print(gs.cv_results_['mean_test_score'])  # 각 교차 검증 평균점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [25]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])  # 평균점수가 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])  # 최고 검증 점수를 만든 매개변수 조합. best_params_와 동일

{'min_impurity_decrease': 0.0001}


## 더 복잡한 매개변수 조합 탐색하기

In [28]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),  # 노드 분할을 위한 불순도 감소 최소량
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)  # 노드 분할을 위한 최소 샘플 수
          }

#### 매개 변수로 수행할 교차 검증 횟수: 9 x 15 x 10 = 1,350개
#### 기분 5-폴드 교차 검증 수행하므로 모델의 수는 1,350 x 5 = 6,750개!

In [29]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)  # 모든 CPU 코어 사용
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [30]:
print(gs.best_params_)  # 최적의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [31]:
print(np.max(gs.cv_results_['mean_test_score']))  # 최고 교차 검증 점수

0.8683865773302731


### 랜덤 서치

In [32]:
from scipy.stats import uniform, randint

In [33]:
rgen = randint(0, 10)  # 0~10
rgen.rvs(10)

array([7, 1, 8, 7, 1, 3, 8, 6, 8, 8])

In [34]:
np.unique(rgen.rvs(1000), return_counts=True)  # 1000개 샘플링해서 각 숫자의 개수 세기

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107, 112,  83, 108, 105, 103, 100,  86,  99,  97]))

In [35]:
ugen = uniform(0, 1)  # 0~1 실수
ugen.rvs(10)

array([0.4339001 , 0.96405033, 0.44742358, 0.5119562 , 0.54567267,
       0.12869766, 0.34358269, 0.66817999, 0.07942191, 0.23071824])

In [36]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),  
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),  # 리프 노드가 되기 위한 최소 샘플의 개수
          }

In [37]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)  # 매개변수 범위에서 100번 샘플링

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd0d070c550>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd0f1eb84c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd0d09db130>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd0d09d8370>},
                   random_state=42)

In [38]:
print(gs.best_params_)  # 최적의 매개변수 조합

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))  # 최고 교차 검증 점수

0.8695428296438884


In [40]:
dt = gs.best_estimator_  # 최적의 모델

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [41]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd0d070c550>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd0f1eb84c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd0d09db130>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd0d09d8370>},
                   random_state=42)

In [42]:
print(gs.best_params_)
print('최고 교차검증 평균점수: ', np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print('최적의 모델의 성능 점수: ', dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
최고 교차검증 평균점수:  0.8458726956392981
최적의 모델의 성능 점수:  0.786923076923077
